# 1. SETTINGS

In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA, FastICA, FactorAnalysis
from sklearn.metrics import log_loss
import copy
import scipy.stats
import os
import time
import pandas as pd

In [ ]:
# helper functions
import functions
from functions import *

In [ ]:
# pandas options
pd.set_option('display.max_columns', None)

In [ ]:
# dark background style
plt.style.use('dark_background')

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# garbage collection
import gc
gc.enable()

# 2. DATA PREPARATION

In [ ]:
# import CSV
df = pd.read_csv('../data/data_v3.csv')
print(df.shape)

In [ ]:
# target variable
target = 'fraud'

In [ ]:
# partitioning
train = df[df[target].isnull() == False]
test  = df[df[target].isnull() == True]
print(train.shape)
print(test.shape)

In [ ]:
# target variable
y = train[target]
del train[target], test[target]

# 3. MODELING

### PARAMETERS

In [ ]:
# drop bad features
excluded_feats = ['id']
features = [f for f in train.columns if f not in excluded_feats]
print(train[features].shape)

In [ ]:
### PARAMETERS

# settings
cores = 12
seed  = 23

# cross-validation
num_folds = 5
shuffle   = True

# muner of rounds
max_rounds = 600
stopping   = 600
verbose    = 200

# LGB parameters
lgb_params = {
    #'objective':          custom_loss,
    #'objective':          'binary:hinge',
    #'objective':          'binary:logistic',
    #'boosting_type':     'gbdt',
    #'objective':         'binary',
    #'metrics':           'logloss',
    'eval_metric':       'logloss',
    'n_estimators' :     [85,100],#[int(i) for i in np.arange(60,120.001,5)], #150,
    #'bagging_fraction':  0.9,
    #'feature_fraction':  0.8,
    'subsample' :         1,#[i for i in np.arange(0.8,1.01,0.025)],#1,
    #'alpha':             [10**i for i in np.arange(0,8.001,1)],#0.1,
    #'lambda':            [2**i for i in np.arange(0,11,1)],#0,
    #'eta':               [10**i for i in np.arange(0,11,1)],
    #'min_split_gain':    0.01,
    'min_child_weight':  0.975,#[i for i in np.arange(0.8,0.86001,0.005)] + [i for i in np.arange(0.92,1.001,0.005)], #0.85,
    #'min_child_samples': 20,

    'silent':            True,
    'verbosity':         -1,
    'learning_rate':     0.31,#0.31,#[i for i in np.arange(0.2,0.5,0.01)],#0.25,
    'max_depth':         6,#[i for i in range(2,12)], #6
    #'max_leaves':        [i for i in np.arange(0,10,1)],#70,
    #'scale_pos_weight':  1,
    #'n_estimators':      max_rounds,
    #'nthread' :          cores,
    #'random_state':      seed,
    'colsample_bytree':  1,#[i for i in np.arange(0.5,1.0001,0.05)], 
    'colsample_bylevel': 1,#[i for i in np.arange(0.5,1.0001,0.05)], 
    'colsample_bynode':  0.6,#[i for i in np.arange(0.5,0.65,0.01)],#0.6,#[i for i in np.arange(0.25,0.75,0.01) if i >=0.36 and i<=0.47 or i >=0.54 and i<=0.64]#0.65,#[i for i in np.arange(0.5,1.0001,0.05)]
    'random_state':      [seed + i*10 for i in range(10)]
}
# data partitinoing

In [ ]:
# placeholders
clfs = []
valid_profit = np.zeros(num_folds) 
preds_test   = np.zeros(test.shape[0])
preds_oof    = np.zeros(train.shape[0])
importances  = pd.DataFrame()

In [ ]:
# SMOTE settings
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
sm = SMOTE(random_state = 23, n_jobs = 10, sampling_strategy = 0.1)
sm2 = SMOTEENN(smote=sm)

### CROSS-VALIDATION

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from imblearn.pipeline import make_pipeline, Pipeline
from collections.abc import Iterable
from sklearn.metrics import make_scorer
from xgboost import XGBClassifier


In [ ]:
qs = [0,100]
scaler_params = {
    #'quantile_range': [(qs[0],i) for i in np.arange(0,101) if i > qs[0]],
    #'quantile_range': [(i,qs[1]) for i in np.arange(0,101,20) if i < qs[1]],
    #'quantile_range': [(qs[0],i) for i in np.arange(qs[1]-5,qs[1]+5,0.1) if i > qs[0]],
    #'quantile_range': [(i,qs[1]) for i in np.arange(qs[0]-5,qs[0]+5,0.1) if i < qs[1]],
    #'with_centering': [True,False],
#    'with_scaling': [True,False]
}
params = dict()
params.update({'clf__'+k: v if isinstance(v,Iterable) and not isinstance(v,str) else [v] for k,v in lgb_params.items()})
#params.update({'rs__'+k:   v if isinstance(v,Iterable) else [v] for k,v in scaler_params.items()})
np.array([len(v) for k,v in params.items()])

In [ ]:
#params['clf__objective'] = [custom_loss3]
rs = RandomizedSearchCV(
    estimator = Pipeline([('rs', RobustScaler()),('sm',sm2),('clf',XGBClassifier())]),
    n_iter = 2000,
    param_distributions = params,
    cv=num_folds, 
    n_jobs=cores,
    verbose=2)
rs.fit(train[features],y)
#df = pd.DataFrame(rs.cv_results_)
variables = [k for k,v in rs.cv_results_.items() if 'param_clf__' in k if len(set(v.data))>1 
             if len(set(v.data)) != 1 and k != 'param_clf__seed' and k != 'param_clf__random_state']
df = pd.DataFrame(rs.cv_results_).groupby(variables).mean().reset_index().sort_values(variables[0])
df2 = pd.DataFrame(rs.cv_results_).groupby(variables).agg(np.std, ddof=1).reset_index().sort_values(variables[0])
df3 = pd.DataFrame(rs.cv_results_).groupby(variables).std().reset_index().sort_values(variables[0])

tmp = {k.replace('param_',''):v for k,v in dict(df.iloc[df['mean_test_score'].argmax()]).items() if 'param_clf__' in k}
params2 = copy.deepcopy(params)
params2.update(tmp)
params2 = {k:v if isinstance(v,Iterable) and not isinstance(v,str) else [v] for k,v in params2.items() if k != 'clf__seed' if k != 'clf__random_state'}

In [ ]:
lgb_params_star = {k.replace('lgbmclassifier__',''):v for k,v in rs.best_params_.items() if 'lgbmclassifier' in k}
print(rs.best_score_, dict(df.iloc[df['mean_test_score'].argmax()][variables]))
import matplotlib.pyplot as plt
plt.plot(df[variables[0]],df['mean_test_score'])
#plt.plot(df2[variables[0]],df2['mean_test_score'])
plt.xlabel(variables[0])
plt.show()
#m = rs.cv_results_['mean_test_score'].max()
#pd.DataFrame(rs.cv_results_['param_clf__min_child_weight'].data[rs.cv_results_['mean_test_score'] >= m]).round(3)
#a = np.arange(0.37,0.8,0.01)
#rs.cv_results_['param_clf__colsample_bynode'].data[rs.cv_results_['mean_test_score'] >= m].max()


In [ ]:
print(pd.Series(rs.best_params_))

In [ ]:
# all arguments that can be passed to the pipeline
#sorted(rs.estimator.get_params().keys())
params2.update({'clf__seed':[3500+i for i in range(500)]})
params2['clf__n_estimators'] = [int(i) for i in params2['clf__n_estimators']]
#params2['clf__min_child_weight'] = [0]

In [ ]:
rs = RandomizedSearchCV(
    estimator = Pipeline([('rs', RobustScaler()),('sm',sm2),('clf',XGBClassifier())]),
    n_iter = 2000,
    param_distributions = params2,
    cv=num_folds, 
    n_jobs=cores,
    verbose=2)
rs.fit(train[features],y)

In [ ]:
### CROSS-VALIDATION LOOP
folds = StratifiedKFold(n_splits = num_folds, random_state = rs.best_params_['clf__seed'], shuffle = shuffle)
for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, y)):
    
    # data partitioning
    trn_x, trn_y = train[features].iloc[trn_idx], y.iloc[trn_idx]
    val_x, val_y = train[features].iloc[val_idx], y.iloc[val_idx]
    
    # scale data
    scaler   = RobustScaler()
    trn_x    = pd.DataFrame(scaler.fit_transform(trn_x),      columns = features)
    val_x    = pd.DataFrame(scaler.transform(val_x),          columns = features)
    tmp_test = pd.DataFrame(scaler.transform(test[features]), columns = features)

    # augment training data with SMOTE
    trn_x, trn_y = sm.fit_sample(trn_x, trn_y)
    trn_x = pd.DataFrame(trn_x, columns = features)
    trn_y = pd.Series(trn_y)
    
    # factor decomposition
    tmp_features = copy.deepcopy(features)
    
    # train lightGBM
    print('Custom early stopping: select the best out of %.0f iterations...' % max_rounds)
    #clf = rs.best_estimator_.get_params()['clf']
    #clf = XGBClassifier(**params2)
    clf = XGBClassifier(**{k.replace('clf__',''):v for k,v in rs.best_params_.items()})
    clf = clf.fit(trn_x, trn_y, 
                  eval_set              = [(trn_x, trn_y), (val_x, val_y)], 
                  eval_metric           = prediction_reward_xgb, 
                  #eval_metric           = "logloss", 
                  #early_stopping_rounds = stopping,
                  verbose               = 0
                 )
    clfs.append(clf)
    
    # find the best iteration
    best_iter = np.argmax(clf.evals_result_['validation_1']['profit']) + 1
    
    # predictions
    preds_oof[val_idx]    = clf.predict_proba(val_x, ntree_limit = best_iter)[:, 1]
    valid_profit[n_fold]  = prediction_reward(val_y, preds_oof[val_idx])[1]
    preds_test           += clf.predict_proba(tmp_test, ntree_limit = best_iter)[:, 1] / folds.n_splits 

    ## importance
    fold_importance_df = pd.DataFrame()
    fold_importance_df['Feature'] = tmp_features
    fold_importance_df['Importance'] = clf.feature_importances_
    fold_importance_df['Fold'] = n_fold + 1
    importances = pd.concat([importances, fold_importance_df], axis = 0)
    
    # print performance
    print('--------------------------------')
    print('FOLD%2d: PROFIT = %.0f' % (n_fold + 1, valid_profit[n_fold]))
    print('--------------------------------')
    print('')
    
    # clear memory
    del trn_x, trn_y, val_x, val_y
    gc.collect()
    
    # uncomment for mean target encoding
    #features = [f for f in train.columns if f not in excluded_feats]
    
    
# print overall performance    
cv_perf = np.sum(valid_profit)
print('--------------------------------')
print('TOTAL PROFIT = %.0f' % cv_perf)
print('--------------------------------')

In [ ]:
##### RECHECK PROFIT  
prediction_reward(y, preds_oof)


###### TRACKING RESULTS (5 folds, strat = True, seed = 23)

# V1: lgb, 5 folds, default features:   80
# V2: add feature:  no. total items:   250
# V3: use logloss for ES, not profit:  260
# V4: add feature: no. weird actions:  275
# V5: custom earlystop for profit:     320
# V6: add SMOTE for minority class:    335
# V7: add robust data scaling:         350 = 95 + 55 + 75 + 35 + 90
# V8: increase learning rate to 0.1:   375 = 95 + 65 + 75 + 50 + 90

In [ ]:
##### VARIABLE IMPORTANCE

# load importance    
top_feats = 100
cols = importances[['Feature', 'Importance']].groupby('Feature').mean().sort_values(by = 'Importance', ascending = False)[0:top_feats].index
importance = importances.loc[importances.Feature.isin(cols)]
    
# plot variable importance
plt.figure(figsize = (10, 6))
sns.barplot(x = 'Importance', y = 'Feature', data = importance.sort_values(by = 'Importance', ascending = False))
plt.tight_layout()

# save plot as pdf
plt.savefig('../var_importance.pdf')

### CUTOFF OPTIMIZATION

In [ ]:
##### OPTIMIZE CUTOFF

# set step
step = 100

# search
cutoffs = []
profits = []
for i in range(0, step):
    cutoffs.append(i / step)
    profits.append(recompute_reward(y, preds_oof, cutoff = cutoffs[i]))
        
# results
plt.figure(figsize = (10,4))
sns.lineplot(x = cutoffs[10:step], y = profits[10:step], color = 'red')
plt.tight_layout()
plt.axvline(x = cutoffs[np.argmax(profits)], color = 'white', linestyle = '--')
print('- optimal cutoff = %.4f' % cutoffs[np.argmax(profits)])
print('- optimal profit = %.4f' % profits[np.argmax(profits)])
plt.savefig('../cutoff_selection.pdf')

# 4. SUBMISSION

In [ ]:
# file name
model = 'xgb_v8'
perf  = str(round(cv_perf, 0).astype('int'))
name  = model + '_' + perf

In [ ]:
# export OOF preds
oof = pd.DataFrame({'id': train['id'], 'fraud': preds_oof})
oof.to_csv('../oof_preds/' + str(name) + '.csv', index = False)
oof.head()

In [ ]:
# check submission
sub = pd.DataFrame({'id': test['id'], 'fraud': preds_test})
sub['fraud'] = np.round(sub['fraud']).astype('int')
sub.head()

In [ ]:
# export submission
sub = sub[['fraud']]
sub.to_csv('../submissions/' + str(name) + '.csv', index = False)
sub.shape

In [ ]:
# check correlation with previous submission
prev_sub = pd.read_csv('../submissions/lgb_v8_375.csv')
cor = np.sum(prev_sub[target] == sub.reset_index()[target]) / len(sub)
print("Share of the same predictions: " + str(np.round(cor, 6)))